In [1]:
pip install audacity-scripting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [audacity-scripting]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import time
from audacity_scripting.bridge import Bridge
from audacity_scripting.utils import run_script

# --- Configuration ---
# IMPORTANT: Adjust these paths for your system
INPUT_AUDIO_PATH = "C:/Users/YourUser/Desktop/my_original_audio.wav" # Windows example
# INPUT_AUDIO_PATH = "/home/youruser/my_original_audio.wav" # Linux example

OUTPUT_AUDIO_PATH = "C:/Users/YourUser/Desktop/my_reversed_segment.wav" # Windows example
# OUTPUT_AUDIO_PATH = "/home/youruser/my_reversed_segment.wav" # Linux example

SEGMENT_START_SECONDS = 5.0  # Start time of the segment to reverse
SEGMENT_END_SECONDS = 10.0   # End time of the segment to reverse

# --- Ensure Audacity is running with mod-script-pipe enabled! ---
# You might need to manually open Audacity first.

def reverse_audio_segment(input_file, output_file, start_sec, end_sec):
    print(f"Attempting to process: {input_file}")

    # Initialize the Audacity Bridge
    # This sets up the named pipe communication
    with Bridge() as bridge:
        # 1. Clear any existing projects (optional, but good for clean start)
        # It's often safer to start with a new project or handle existing one.
        # For this simple example, we assume Audacity is relatively clean or will create a new project.
        # However, Audacity scripting usually works on the *currently open* project.
        # Let's try to ensure a new project for clarity:
        run_script(bridge, "New:")
        time.sleep(0.5) # Give Audacity a moment

        # 2. Import the audio file
        import_command = f'Import2: Filename="{input_file}"'
        print(f"Sending: {import_command}")
        response = run_script(bridge, import_command)
        print(f"Import response: {response}")
        time.sleep(1) # Give Audacity time to load the audio

        # 3. Select the desired segment
        select_command = f'SelectTime: Start={start_sec} End={end_sec}'
        print(f"Sending: {select_command}")
        response = run_script(bridge, select_command)
        print(f"Select response: {response}")
        time.sleep(0.5) # Give Audacity a moment

        # 4. Apply the Reverse effect
        print("Sending: Reverse:")
        response = run_script(bridge, "Reverse:")
        print(f"Reverse response: {response}")
        time.sleep(1) # Give Audacity time to process the effect

        # 5. Export the modified audio
        # Note: Export2 saves the *entire project* as a new file.
        # If you only wanted the reversed segment, you'd need to copy it to a new track first,
        # then export only that new track, or use other Audacity commands to achieve that.
        # For simplicity here, we'll export the whole (now partially reversed) track.
        export_command = f'Export2: Filename="{output_file}"'
        print(f"Sending: {export_command}")
        response = run_script(bridge, export_command)
        print(f"Export response: {response}")
        time.sleep(1) # Give Audacity time to export

        # 6. Close the project without saving changes to the original .aup3 (optional)
        print("Sending: Close: Project=1") # Close current project, 1 to not save changes
        response = run_script(bridge, "Close: Project=1")
        print(f"Close response: {response}")

    print(f"Script finished. Check '{output_file}' for the result.")

if __name__ == "__main__":
    if not os.path.exists(INPUT_AUDIO_PATH):
        print(f"Error: Input audio file not found at '{INPUT_AUDIO_PATH}'. Please update the path.")
    else:
        reverse_audio_segment(INPUT_AUDIO_PATH, OUTPUT_AUDIO_PATH, SEGMENT_START_SECONDS, SEGMENT_END_SECONDS)